In [1]:
from rpy2.robjects import r
import rpy2.robjects as robjects
import sys
from datetime import datetime

In [2]:
import pandas as pd
df = pd.read_csv("result_survival.csv", encoding="UTF-8")

In [3]:
for i in range(len(df)):
    df.loc[i, 'establish_date_str']=str(df.loc[i,'establish_date']).split('.')[0]

In [4]:
df.head()

,ked_code,business_number,establish_date,avg_credit_payment_ratio,avg_credit_payment_period,credit_grade,suspension_date,avg_working_year,avg_employee_count,avg_rent_deposite,...,listing_YN,recent_date,accounts_receivable,asset,debt,net_income,non_current_liabilities,operating_profit,sales,establish_date_str
0,9010609986,3110384463,20111104.0,NaN,NaN,18.0,NaN,0.0,NaN,0.0,...,Y,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20111104
1,2624117,6029074217,20140520.0,0.0,0.0,NaN,NaN,NaN,NaN,0.0,...,Y,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20140520
2,9011039453,4102937030,19861212.0,NaN,NaN,NaN,NaN,0.0,NaN,0.0,...,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,19861212
3,1388321,1200866035,20051109.0,NaN,NaN,11.0,NaN,0.0,3.0,0.0,...,Y,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20051109
4,720349,1221037451,20030728.0,0.0,19.4,22.0,NaN,0.0,3.5,0.0,...,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20030728


In [5]:
df = df[['ked_code', 'business_number', 'establish_date_str', 'share_holders_count', 'accounts_receivable', 'suspension_date']]

In [6]:
df['bankruptcy']=df['suspension_date'].notnull()

In [7]:
df=df[['ked_code', 'business_number', 'establish_date_str', 'share_holders_count', 'accounts_receivable', 'bankruptcy']]

In [8]:
df=df.dropna(axis=0)

In [9]:
result = pd.DataFrame(columns=['ked_code', 'predict', 'bankruptcy'])

In [10]:
zipped_df = zip(range(len(df)),df['ked_code'],df['establish_date_str'], df['share_holders_count'], df['accounts_receivable'], df['bankruptcy'])

In [11]:
# r에서의 모듈 임포트
r("library(KMsurv)")
r("library(survival)")
r("library(ggplot2)")
r("library(survminer)")
r("library(car)")
r("library(maxstat)")
r("library(party)")
for company in (list(zipped_df)):
    share_holders_count = company[3]
    accounts_receivable = company[4]
    established_date_string = company[2]
    # 쿼리가 완료된 csv 파일 로드
    r("df = read.csv('result_survival.csv',header=T)")

    # 전처리
    r("df[,\"failure\"] = ifelse(is.na(df$suspension_date),0,1)")
    r("df$suspension_date = ifelse(is.na(df$suspension_date),\"20200205\",df$suspension_date)")
    r("df$establish_date=as.character(df$establish_date)")
    r("df[,\"time\"] = as.integer(as.Date(df$suspension_date,format=\"%Y%m%d\")-as.Date(df$establish_date,format = \"%Y%m%d\"))")
    r("data=df[,c(\"ked_code\",\"business_number\",\"debt\",\"accounts_receivable\",\"avg_credit_payment_ratio\",\"share_holders_count\",\"non_current_liabilities\",\"operating_profit\",\"failure\",\"time\")]")
    r("data<-na.omit(data)")
    r("data2 <- df[,c(\"ked_code\",\"business_number\",\"accounts_receivable\",\"share_holders_count\",\"failure\",\"time\")]")
    r("data2 <- na.omit(data2)")
    r("test<-coxph(Surv(time,failure)~share_holders_count+accounts_receivable,data=data2)")
    r("predict<-survfit(test,c(share_holders_count=" + str(share_holders_count) + ",accounts_receivable=" + str(accounts_receivable) + "))")


    predict = robjects.r['predict']
    today = datetime.today()
    established_date = datetime.strptime(established_date_string, '%Y%m%d')
    operation_days = (today-established_date).days

    times = predict[1]
    survs = predict[5]

    min_diff = sys.maxsize
    min_index = -1
    surv_probability="측정 불가"
    for i in range(len(times)):
        diff = abs(times[i]-operation_days)
        if(diff<min_diff):
            min_diff = diff
            min_index = i
    if(min_index==-1):
        continue
    else:
        surv_probability = survs[min_index]
    row = [company[1], surv_probability, company[5]]
    result.loc[len(result)]=row

R[write to console]: 필요한 패키지를 로딩중입니다: ggpubr

R[write to console]: 필요한 패키지를 로딩중입니다: magrittr

R[write to console]: 필요한 패키지를 로딩중입니다: carData

R[write to console]: 필요한 패키지를 로딩중입니다: grid

R[write to console]: 필요한 패키지를 로딩중입니다: mvtnorm

R[write to console]: 필요한 패키지를 로딩중입니다: modeltools

R[write to console]: 필요한 패키지를 로딩중입니다: stats4

R[write to console]: 
다음의 패키지를 부착합니다: ‘modeltools’


R[write to console]: The following object is masked from ‘package:car’:

    Predict


R[write to console]: 필요한 패키지를 로딩중입니다: strucchange

R[write to console]: 필요한 패키지를 로딩중입니다: zoo

R[write to console]: 
다음의 패키지를 부착합니다: ‘zoo’


R[write to console]: The following objects are masked from ‘package:base’:

    as.Date, as.Date.numeric


R[write to console]: 필요한 패키지를 로딩중입니다: sandwich



In [12]:
result.head()

,ked_code,predict,bankruptcy
0,211080,0.575896,False
1,258477,0.529004,False
2,602757,0.731473,False
3,504204,0.061190,False
4,245617,0.170900,False


In [13]:
# 이렇게 평균값으로 해 봤을 때, 정확도 36%..개망
result['predict'].mean()

0.6747607500252583

In [14]:
result[result.bankruptcy==True].count()['bankruptcy']

293

In [15]:
def hit_ratio(result):
    predict_critic = result['predict'].mean()
    predict = result['predict']
    bankruptcy = result['bankruptcy']
    
    true_true = 0
    true_false = 0
    false_true = 0
    false_false = 0
    
    for i in range(len(predict)):
        # 이게 파산한다고 예측한 경우. 생존 확률이 평균보다 작다.
        if(predict[i]<predict_critic):
            # 근데 파산한 경우
            if(bankruptcy[i]==True):
                true_true = true_true + 1
            # 근데 파산하지 않은 경우
            else:
                false_true = false_true + 1
        # 이게 파산하지 않는다고 예측한 경우
        else:
            if(bankruptcy[i]==True):
                true_false = true_false + 1
            else:
                false_false = false_false + 1
    
    hit_ratio_result = (true_true+false_false)/len(result)
    
    print("--------------정확도 측정 결과--------------")
    print("      예측          True        False")
    print("실제")
    print("True               " + str(true_true) + "          " + str(true_false))
    print("False              " + str(false_true) + "          " + str(false_false))
    print("hit ratio는 " + str(true_true+false_false) + " / " + str(len(result)) + " = " + str(hit_ratio_result) + "% 입니다.")

In [16]:
hit_ratio(result)

--------------정확도 측정 결과--------------
      예측          True        False
실제
True               178          115
False              107          214
hit ratio는 392 / 614 = 0.6384364820846905% 입니다.
